In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from pandas.testing import assert_frame_equal
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
p1 = Path.cwd() / 'back_data'

In [2]:
# Pandas 객체 병합은 append() 메서드(곧 deprecate 예정), concat() 함수, join() 메서드, merge() 메서드 등 존재

In [3]:
names = pd.read_csv(p1 / 'names.csv')
names

,Name,Age
0,Cornelia,70
1,Abbas,69
2,Penelope,4
3,Niko,2


In [4]:
# loc 속성을 통해 새로운 행 추가 가능(단, 원시 DataFrame 변형 발생) -> 리스트, 딕셔너리, Series 등 활용 가능
# ex. names.loc[len(name)] = pd.Series({'Name':'Dean', 'Age':32}))

In [7]:
# append() 메서드를 활용할 경우 ignore_index=True 인자 꼭 전달해야
(names
.append({'Name':'Aria', 'Age':1}, ignore_index=True)
# 그게 아니라면 전달하는 Series에서 name 파라미터를 필수로 활용해야
# Series는 오직 하나의 열만 있는 개념. 즉 index와 columns가 아님 -> 그래서 name 파라미터를 쓴다고 생각해야
.append(pd.Series({'Name':'Zach', 'Age':3}, name=len(names)))
)

C:\Users\admin\AppData\Local\Temp\ipykernel_19236\1346535784.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  (names
C:\Users\admin\AppData\Local\Temp\ipykernel_19236\1346535784.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  (names


,Name,Age
0,Cornelia,70
1,Abbas,69
2,Penelope,4
3,Niko,2
4,Aria,1
4,Zach,3


In [8]:
bball_16 = pd.read_csv(p1 / 'baseball16.csv')
bball_16.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,altuvjo01,2016,1,HOU,AL,161,640,108,216,42,...,96.0,30.0,10.0,60,70.0,11.0,7.0,3.0,7.0,15.0
1,bregmal01,2016,1,HOU,AL,49,201,31,53,13,...,34.0,2.0,0.0,15,52.0,0.0,0.0,0.0,1.0,1.0
2,castrja01,2016,1,HOU,AL,113,329,41,69,16,...,32.0,2.0,1.0,45,123.0,0.0,1.0,1.0,0.0,9.0
3,correca01,2016,1,HOU,AL,153,577,76,158,36,...,96.0,13.0,3.0,75,139.0,5.0,5.0,0.0,3.0,12.0
4,gattiev01,2016,1,HOU,AL,128,447,58,112,19,...,72.0,2.0,1.0,43,127.0,6.0,4.0,0.0,5.0,12.0


In [12]:
# to_dict() 메서드를 통해 예제 행을 딕셔너리 형태로 추출
data_dict = bball_16.iloc[0].to_dict()
# dictionary comprehension 이용하여 딕셔너리의 값을 다 제거(문자열을 경우 공백, 숫자일 경우 np.nan 형태)
# isinstance() 활용하거나 혹은 type() 함수 활용도 가능 (if type(v) == str)
new_data_dict = {k:'' if isinstance(v, str) else np.nan for k, v in data_dict.items()}
new_data_dict

{'playerID': '',
 'yearID': nan,
 'stint': nan,
 'teamID': '',
 'lgID': '',
 'G': nan,
 'AB': nan,
 'R': nan,
 'H': nan,
 '2B': nan,
 '3B': nan,
 'HR': nan,
 'RBI': nan,
 'SB': nan,
 'CS': nan,
 'BB': nan,
 'SO': nan,
 'IBB': nan,
 'HBP': nan,
 'SH': nan,
 'SF': nan,
 'GIDP': nan}

In [13]:
# DataFrame에 단일 행을 추가하는 것은 비싼 작업 -> 단일 행 추가로 루프(순환)을 진행한다면 잘못된 행위
# append() 메서드 활용할 경우 하나의 행씩 추가하는 게 아니라, 그냥 전체 행의 리스트 자체를 append() 메서드에 삽입
# 내부적으로 Pandas는 Series 리스트를 단일 DataFrame으로 변환한 다음 데이터를 추가

In [21]:
# concat() 함수를 사용하여 두 개 이상의 DataFrame 혹은 Series를 세로와 가로로 함께 연결
# list comprehension 이용하여 stocks_2016, stocks_2017 DataFrame을 stock_list에 저장
stock_list = [pd.read_csv(p1 / f'stocks_{year}.csv', index_col='Symbol') for year in [2016, 2017]]
# pd.concat() 함수 활용 -> keys, names 파라미터 가능
pd.concat(stock_list, keys=[2016, 2017], names=['Year', 'Symbol'])
# concat() 함수는 가로 연결 시 기본적으로 outer join 활용 -> join 파라미터를 통해 변경
pd.concat(stock_list, keys=[2016, 2017], join='inner', names=['Year', None], axis=1)

Year     2016            2017          
       Shares Low High Shares  Low High
Symbol                                 
AAPL       80  95  110     50  120  140
TSLA       50  80  130    100  100  300

In [22]:
# concat() 함수 : 수직/수평 병합, 인덱스에 대해서만 정렬, 인덱스에 중복이 있으면 오류 발생, 기본 설정은 outer join
# join() 메서드 : 수평 병합, 호출 DataFrame의 열/인덱스를 다른 객체의 인덱스에 대해 정렬, 중복 시 카티션 곱, 기본 설정은 left join
# merge() 메서드 : 수평 병합, 호출 DataFrame의 열/인덱스를 다른 객체의 열/인덱스에 대해 정렬, 중복시 카티션 곱, 기본 설정은 inner join

In [23]:
years = [2016, 2017, 2018]
stock_tables = [pd.read_csv(p1 / f'stocks_{year}.csv', index_col='Symbol') for year in years]
stock_2016, stock_2017, stock_2018 = stock_tables

In [25]:
(stock_2016
# lsuffix, rsuffix 파라미터 이용하여 중복 열 이름 조정
.join(stock_2017, lsuffix='_2016', rsuffix='_2017', how='outer')
# 혹은 DataFrame에서 add_suffix() 메서드 이용하여 이름 조정 후 진행
.join(stock_2018.add_suffix('_2018'), how='outer')
)

,Shares_2016,Low_2016,High_2016,Shares_2017,Low_2017,High_2017,Shares_2018,Low_2018,High_2018
Symbol,,,,,,,,,
AAPL,80.0,95.0,110.0,50.0,120.0,140.0,40.0,135.0,170.0
AMZN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,900.0,1125.0
GE,NaN,NaN,NaN,100.0,30.0,40.0,NaN,NaN,NaN
IBM,NaN,NaN,NaN,87.0,75.0,95.0,NaN,NaN,NaN
SLB,NaN,NaN,NaN,20.0,55.0,85.0,NaN,NaN,NaN
TSLA,50.0,80.0,130.0,100.0,100.0,300.0,50.0,220.0,400.0
TXN,NaN,NaN,NaN,500.0,15.0,23.0,NaN,NaN,NaN
WMT,40.0,55.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# 아예 DataFrame의 열 이름을 add_suffix() 메서드로 변경한 후 두 DataFrame을 리스트로 만들어서
other = [stock_2017.add_suffix('_2017'), stock_2018.add_suffix('_2018')]
(stock_2016.add_suffix('_2016')
# 해당 리스트를 join() 메서드에 전달하는 것도 가능(단, 순서는 변경됨 -> sort_index()를 통한 정렬 필요)
.join(other, how='outer')
)

,Shares_2016,Low_2016,High_2016,Shares_2017,Low_2017,High_2017,Shares_2018,Low_2018,High_2018
Symbol,,,,,,,,,
AAPL,80.0,95.0,110.0,50.0,120.0,140.0,40.0,135.0,170.0
TSLA,50.0,80.0,130.0,100.0,100.0,300.0,50.0,220.0,400.0
WMT,40.0,55.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN
GE,NaN,NaN,NaN,100.0,30.0,40.0,NaN,NaN,NaN
IBM,NaN,NaN,NaN,87.0,75.0,95.0,NaN,NaN,NaN
SLB,NaN,NaN,NaN,20.0,55.0,85.0,NaN,NaN,NaN
TXN,NaN,NaN,NaN,500.0,15.0,23.0,NaN,NaN,NaN
AMZN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,900.0,1125.0


In [34]:
# concat() 함수를 사용하여 복제 -> join 파라미터
(pd.concat(stock_tables, keys=years, axis=1, join='outer')
# 두 단계인 열 레벨을 한 단계로 조정 필요
.swaplevel(0, 1, axis=1)
# set_axis() 메서드 이용하여 DataFrame의 열 레벨 변경 가능
.pipe(lambda df: df.set_axis(df.columns.to_flat_index(), axis=1))
# rename() 메서드 활용하여 열 이름 변경 : lambda 함수 지정 -> label은 '하나의 열'이므로 list comprehension은 '_'.join() 안에 들어가야
.rename(columns=lambda label: '_'.join([str(col) for col in label]))
)

,Shares_2016,Low_2016,High_2016,Shares_2017,Low_2017,High_2017,Shares_2018,Low_2018,High_2018
Symbol,,,,,,,,,
AAPL,80.0,95.0,110.0,50.0,120.0,140.0,40.0,135.0,170.0
TSLA,50.0,80.0,130.0,100.0,100.0,300.0,50.0,220.0,400.0
WMT,40.0,55.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN
GE,NaN,NaN,NaN,100.0,30.0,40.0,NaN,NaN,NaN
IBM,NaN,NaN,NaN,87.0,75.0,95.0,NaN,NaN,NaN
SLB,NaN,NaN,NaN,20.0,55.0,85.0,NaN,NaN,NaN
TXN,NaN,NaN,NaN,500.0,15.0,23.0,NaN,NaN,NaN
AMZN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,900.0,1125.0
